In [49]:
import math
import pandas as pd
#variables

#####VARIABLES#######
boltDia = 0.625
shearStrength = 68 #this is the value for an A325 bolt with threads not in the shear plane 
numBolts = 3
boltSpacing = 3
e = 4  #this is the distance from the primary beam face to the CG of the bolt group

In [3]:
boltArea = (boltDia/2)**2*math.pi
phi=0.75 #this is the safety factor for bolt shear for the LRFD method
R_ult = round(phi*boltArea*shearStrength,3) # this is the design shear strength of a single bolt

print(f"The design strength of a single bolt is: {R_ult}")
print("This value will be used as R_ult\n")

deltaMax = 0.34 #maximum bolt deformation per AISC 7-13

#determine location of bolts
bgHeight = boltSpacing * (numBolts-1)

boltCord = [] #starting from the top bolt

i = 0 
print(f"bolt coordinates with the y value of the center of the bolt group CG being 0:")
while i < numBolts:
    boltTuple = (0,bgHeight/2-boltSpacing*i)
    boltCord.append(boltTuple)
    print(f"bolt: {i+1} = {boltTuple}")
    i+=1


The desing strength of a single bolt is: 15.647
This value will be used as R_ult

bolt coordinates with the y value of the center of the bolt group CG being 0:
bolt: 1 = (0, 3.0)
bolt: 2 = (0, 0.0)
bolt: 3 = (0, -3.0)


In [63]:
#initial Ix and Iy assume the IC is at the vertical center of the bolt group (y=0)



####variables to be used in the calculations
c = 0 #distance from the IC to the individual bolt
cMax = 0 #distance of bolt furtherst away from the IC
cx = 0 #x distance from IC to the individual bolt
cy = 0 #y distance from IC to the individual bolt
R = 0 #Calculated strength on individual bolt
Rx = 0 #Caluclated strength in the x direction of an individual bolt
Ry = 0 #Calculated strength in the y direction of an individual bolt
mIC = 0 #moment value of the bolts around IC
sumMIC = 0 #total mIC for all bolts
delta = 0 #bolt deformation of individual bolt
deltaMax = 0.34 #maximum deformation of bolt before failur from AISC manual
icx = 0.5 #variable x coordinate for the IC
icy = 0  #this will be 0 for all vertical loads because we centered our bolt group vertically to our IC
ic = (icx,icy) #coordinates to the IC
p_ulty = 0 #total vertical load the bolt group can resist 
mP = 0 #moment calculated shear strength (p_ulty) of the bolt group around IC


iterations = 1000 #number of times trying to find ic
iteration = 0
while iteration<iterations:
    ic=(icx,icy)
    cList = []
    for bolt in boltCord:
        c = math.dist(ic,bolt)
        cList.append(c)

    cMax = max(cList)

    #locations in bolt data list for each type of data

    boltData = pd.DataFrame(columns=["Bolt Number",
                                     "c",
                                     "cx",
                                     "cy",
                                     "delta",
                                     "R",
                                     "rx",
                                     "ry",
                                     "mIC"])
    
    boltNum = 1  #we don't want a bolt number 0
    for bolt in boltCord:
        c = math.dist(ic,bolt) #distance from the IC to the individual bolt
        cx = icx
        cy = bolt[1] #equal to the y value of the bolt coordinate
        
        delta = deltaMax*(c/cMax) #the delta of an individual bolt is linearly relative to deltaMax and cMax 
        
        R = R_ult*math.pow(1-math.pow(math.e,-10*delta),0.55)

        
        rx = -R*(cy/c) 
        ry = R*(cx/c)

        mIC = -rx*cy+ry*cx #get the moment of bolt reactions aroudn the IC

        boltData.loc[boltNum-1]=[int(boltNum),round(c,2),round(cx,2),round(cy,2),round(delta,2),round(R,2),round(rx,2),round(ry,2),round(mIC,2)]
        boltNum+=1


    sumMIC = boltData["mIC"].sum()
    p_ulty = boltData["ry"].sum() 
    mP = (e+icx)*p_ulty
    
    mDiff = sumMIC-mP   #the difference in the moment of the bolt reactions around IC and the Load around I

    #convergence = abs(mDiff)/max(abs(sumMIC),abs(mP))
    #print(f"convergence = {convergence*100}%")
    print(f"difference in moment = {mDiff}")
    
    if max(abs(sumMIC),abs(mP))*0.01>abs(mDiff): #converge within 1% of moments
        print("solution found")
        iteration = iterations
    else:
        icx+=mDiff*0.001
        
    #print("\n")
    #print(boltData)
    #print(f"iteration: {iteration}")
    #print(f"Total moment of bolt reactions around the IC = {round(sumMIC,2)}")
    #print(f"Total shear force the bolt groud can resist = {round(p_ulty,2)}")
    #print(f"Moment from the calculated shear strength of the calculation aroudn IC = {round(mP,2)}")
    #print("\n\n\n")
            
    iteration+=1

print(boltData)
print(f"Total moment of bolt reactions around the IC = {round(sumMIC,2)}")
print(f"Total shear force the bolt groud can resist = {round(p_ulty,2)}")
print(f"Moment from the calculated shear strength of the calculation aroudn IC = {round(mP,2)}")
print(f"Location of IC: ({round(icx,3)},{icy})")
    
        

difference in moment = 31.50500000000001
difference in moment = 29.089507949999984
difference in moment = 26.976547323594986
difference in moment = 25.03321324627673
difference in moment = 23.27887850229216
difference in moment = 21.673680522980547
difference in moment = 20.132482222786322
difference in moment = 18.844237858521552
difference in moment = 17.567569465342544
difference in moment = 16.31052652370738
difference in moment = 15.34086401093245
difference in moment = 14.34222543185183
difference in moment = 13.413401966553721
difference in moment = 12.55184905157381
difference in moment = 11.739739308121372
difference in moment = 10.908971402300011
difference in moment = 10.244612886595078
difference in moment = 9.704724508066079
difference in moment = 9.018272257979149
difference in moment = 8.477836569899736
difference in moment = 7.964609434430798
difference in moment = 7.401389365990724
difference in moment = 6.993604349278243
difference in moment = 6.582838519622129
differ